In [17]:
import sys
sys.path.append('../..')
import pandas as pd
import backtrader as bt
from datetime import datetime
import matplotlib.pyplot as plt
from Tools.backtest_tools import *
import plotly.graph_objects as go
import pandas_ta as ta 
from Tools.plotting import CandlePlot

In [18]:
# Load the data and convert the timestamp to datetime
df = pd.read_csv('../../DB/5m/SEIUSDT_5m.csv', parse_dates=True) 

# df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
# # # Convert 'date' to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])  # Assuming the 'date' is in milliseconds

# # # Set 'date' as index
df.set_index('datetime', inplace=True)

# df.iloc[0]
# df.dtypes
# df.head()
df_t = df[:].copy()

In [19]:
class BBands:
    def __init__(
            self, 
            df, 
            type=['long', 'short'],
            slow_ema_w = 50,
            fast_ema_w = 30,
            rsi_w = 10,
            bb_w = 15,
            std_w=1.5,
            ATR_w=7

            ) -> None:
        
        self.df = df
        self.use_long = True if "long" in type else False
        self.use_short = True if "short" in type else False
        self.slow_ema_w = 50
        self.fast_ema_w = 30
        self.rsi_w = 10
        self.bb_w = 15
        self.std_w=1.5
        self.ATR_w=7

    def populate_indicators(self):
        # -- Clear dataset --
        df = self.df
        df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)

        df["open_long_market"] = False
        df["close_long_market"] = False
        df["open_short_market"] = False
        df["close_short_market"] = False

        
        # -- Populate indicators --
        df['slow_ema'] = ta.ema(self.df.close, length=self.slow_ema_w)
        df['fast_ema'] = ta.ema(self.df.close, length=self.fast_ema_w)
        df['rsi'] = ta.rsi(self.df.close, length=self.rsi_w)
        m_bbands = ta.bbands(self.df.close, length=self.bb_w, std=self.std_w) 
        df['ATR']= ta.atr(self.df.high, self.df.low, self.df.close, length=self.ATR_w)

        df = df.join(m_bbands)
        
        # df = get_n_columns(df, ["lvl_above",  "lvl_below", "close"], 1)
        
        self.df = df    
        return self.df

    def populate_buy_sell(self): 
        df = self.df
        # -- Initiate populate --
        if self.use_long:
            # -- Populate open long market --
            df.loc[
                (df['fast_ema'] > df['slow_ema']) 
                & (df['close'] < df['BBL_15_1.5'])                 
                , "open_long_market"
            ] = True
            
            
            # -- Populate close long market --
            df.loc[
                (df['close'] > df['BBU_15_1.5']) 
                , f"close_long_market"
            ] = True

        if self.use_short:
            # -- Populate open short market --
            df.loc[
                (df['fast_ema'] < df['slow_ema']) 
                & (df['close'] > df['BBU_15_1.5'])                 
                , "open_short_market"
            ] = True
            
            # -- Populate close short market --
            df.loc[
                (df['close'] < df['BBL_15_1.5']) 
                , f"close_short_market"
            ] = True
               
        self.df = df   
        return self.df
    

    # Running the backtest
    
    def run_backtest(self, initial_wallet=1000, leverage=1):
        df = self.df[:]
        wallet = initial_wallet
        maker_fee = 0.0002
        taker_fee = 0.0007
        trades = []
        days = []
        current_day = 0
        previous_day = 0
        current_position = None

        for index, row in df.iterrows():
            
            # -- Add daily report --
            current_day = index.day
            if previous_day != current_day:
                temp_wallet = wallet
                if current_position:
                    if current_position['side'] == "LONG":
                        close_price = row['close']
                        trade_result = (close_price - current_position['price']) / current_position['price']
                        temp_wallet += temp_wallet * trade_result
                        fee = temp_wallet * taker_fee
                        temp_wallet -= fee
                    elif current_position['side'] == "SHORT":
                        close_price = row['close']
                        trade_result = (current_position['price'] - close_price) / current_position['price']
                        temp_wallet += temp_wallet * trade_result
                        fee = temp_wallet * taker_fee
                        temp_wallet -= fee
                    
                days.append({
                    "day":str(index.year)+"-"+str(index.month)+"-"+str(index.day),
                    "wallet":temp_wallet,
                    "price":row['close']
                })
            previous_day = current_day
            if current_position:
            # -- Check for closing position --
                if current_position['side'] == "LONG":                     
                    # -- Close LONG market --
                    if row['close_long_market']:
                        close_price = row['close']
                        trade_result = ((close_price - current_position['price']) / current_position['price']) * leverage
                        wallet += wallet * trade_result
                        fee = wallet * taker_fee
                        wallet -= fee
                        trades.append({
                            "open_date": current_position['date'],
                            "close_date": index,
                            "position": "LONG",
                            "open_reason": current_position['reason'],
                            "close_reason": "Market",
                            "open_price": current_position['price'],
                            "close_price": close_price,
                            "open_fee": current_position['fee'],
                            "close_fee": fee,
                            "open_trade_size":current_position['size'],
                            "close_trade_size": wallet,
                            "wallet": wallet
                        })
                        current_position = None
                        
                elif current_position['side'] == "SHORT":
                    # -- Close SHORT Market --
                    if row['close_short_market']:
                        close_price = row['close']
                        trade_result = ((current_position['price'] - close_price) / current_position['price']) * leverage
                        wallet += wallet * trade_result
                        fee = wallet * taker_fee
                        wallet -= fee
                        trades.append({
                            "open_date": current_position['date'],
                            "close_date": index,
                            "position": "SHORT",
                            "open_reason": current_position['reason'],
                            "close_reason": "Market",
                            "open_price": current_position['price'],
                            "close_price": close_price,
                            "open_fee": current_position['fee'],
                            "close_fee": fee,
                            "open_trade_size": current_position['size'],
                            "close_trade_size": wallet,
                            "wallet": wallet
                        })
                        current_position = None

            # -- Check for opening position --
            else:
                # -- Open long Market --
                if row['open_long_market']:
                    open_price = row['close']
                    fee = wallet * taker_fee
                    wallet -= fee
                    pos_size = wallet
                    current_position = {
                        "size": pos_size,
                        "date": index,
                        "price": open_price,
                        "fee":fee,
                        "reason": "Market",
                        "side": "LONG",
                    }
                elif row['open_short_market']:
                    open_price = row['close']
                    fee = wallet * taker_fee
                    wallet -= fee
                    pos_size = wallet
                    current_position = {
                        "size": pos_size,
                        "date": index,
                        "price": open_price,
                        "fee":fee,
                        "reason": "Market",
                        "side": "SHORT"
                    }
                    
                    
        df_days = pd.DataFrame(days)
        df_days['day'] = pd.to_datetime(df_days['day'])
        df_days = df_days.set_index(df_days['day'])

        df_trades = pd.DataFrame(trades)
        if df_trades.empty:
            print("!!! No trades")
            return None
        else:
            df_trades['open_date'] = pd.to_datetime(df_trades['open_date'])
            df_trades = df_trades.set_index(df_trades['open_date'])  
            self.trades = df_trades
        
        return get_metrics(df_trades, df_days) | {
            "wallet": wallet,
            "trades": df_trades,
            "days": df_days
        }

    def pointpos():
        pass

        

    def plot(self):
            # plot_df = self.df[1000:].copy()
            # plt.figure(figsize=(18, 6))
            # plt.plot(plot_df['close'], label='Close Price')
            # plt.plot(plot_df['ma_band'], label='Moving Average')
            # plt.plot(plot_df['lvl_above'], label='BB Upper')
            # plt.plot(plot_df['lvl_below'], label='BB Lower')
            # plt.legend()
            # plt.show()
            
            cp = CandlePlot(self.df.iloc[-3000:])
            cp.add_traces(['slow_ema', 'fast_ema', 'BBL_15_1.5', 'BBU_15_1.5', 'BBM_15_1.5'])
            cp.show_plot()
        

In [20]:
strat = BBands(df_t)

In [ ]:
strat.populate_indicators()

In [ ]:
strat.populate_buy_sell()

In [23]:
cp = CandlePlot(strat.df)


In [ ]:
strat.populate_indicators()
strat.populate_buy_sell()
# strat.trades
bt_result = strat.run_backtest(initial_wallet=1000, leverage=1)
df_trades, df_days = basic_single_asset_backtest(trades=bt_result['trades'], days=bt_result['days'])


In [ ]:
plot_wallet_vs_asset(df_days=df_days)

In [26]:
# plot_futur_simulations(
#     df_trades=df_trades,
#     trades_multiplier=1,
#     trades_to_forecast=1000,
#     number_of_simulations=100,
#     true_trades_to_show=10,
#     show_all_simulations=True,
# )

In [ ]:
plot_bar_by_month(df_days=df_days)